# Project Phase 3: Video Moment Detection

## Connection to our index on OpenSearch

In [4]:
import pprint as pp
import requests

host = 'localhost'
port = 9200

user = 'admin' # Add your user name here.
password = 'a.Zerty12.3' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [5]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = (user, password),
    use_ssl = True,
    verify_certs = False,
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")

/home/marc/anaconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/opensearchpy/connection/http_urllib3.py:214: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/home/marc/anaconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'admin': {'settings': {'index': {'creation_date': '1744387011382',
                                  'knn': 'true',
                                  'number_of_replicas': '0',
                                  'number_of_shards': '4',
                                  'provided_name': 'admin',
                                  'refresh_interval': '-1',
                                  'replication': {'type': 'DOCUMENT'},
                                  'uuid': '3a5NHwlzQmi4RmFEhyf33g',
                                  'version': {'created': '136347827'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'admin': {'mappings': {'properties': {'caption_bow': {'analyzer': 'standard',
                                                       'type': 'text'},
                           

/home/marc/anaconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/marc/anaconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/marc/anaconda3/envs/nlp-cv-ir/lib/python3.9/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/marc/an

## Video downloading

In [4]:
import os
import subprocess

# Directory where videos will be saved
download_path = "./videos"
os.makedirs(download_path, exist_ok=True)  # Create folder if it doesn't exist

video_url = "https://youtu.be/daxodF_Pnzg?si=GM0lJFcjXopRpQ2c" # Replace with the actual video URL
video_id = video_url.split("=")[1]  # Extracted video ID from the URL
#print(f"Video ID: {video_id}")

# Loop through each video in the top 10
output_template = os.path.join(download_path, f"{video_id}.mp4")

try:
    print(f"Downloading: {video_url}")
    subprocess.run(["yt-dlp", "-o", output_template, video_url], check=True)
    print(f"Downloaded: {output_template}")
except subprocess.CalledProcessError:
    print(f"Failed to download {video_id}.")

Downloading: https://youtu.be/daxodF_Pnzg?si=GM0lJFcjXopRpQ2c
[youtube] Extracting URL: https://youtu.be/daxodF_Pnzg?si=GM0lJFcjXopRpQ2c
[youtube] daxodF_Pnzg: Downloading webpage
[youtube] daxodF_Pnzg: Downloading tv client config
[youtube] daxodF_Pnzg: Downloading player 59b252b9
[youtube] daxodF_Pnzg: Downloading tv player API JSON
[youtube] daxodF_Pnzg: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js
         n = kIe99w33WYcDc0M0Ihb3 ; player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js
         n = fTDuR_TSCqtMIfrhLNQi ; player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js
         n = Ec65oaq00xkXNoBlD40k ; player = https://www.youtube.com/s/player/59b252b9/player_ias.vflset/en_US/base.js


[youtube] daxodF_Pnzg: Downloading m3u8 information
[info] daxodF_Pnzg: Downloading 1 format(s): 616+234
[download] ./videos/GM0lJFcjXopRpQ2c.mp4 has already been downloaded
Downloaded: ./videos/GM0lJFcjXopRpQ2c.mp4


## Frames extraction

In [3]:
import av
import av.datasets
import os

output_folder = "./frames_part3"
video = "./videos/6gyD-Mte2ZM.mp4"
# File name without extension
filename = os.path.splitext(video)[0]
if not os.path.isdir(output_folder + "/" + filename):
    os.makedirs(output_folder + "/" + filename)
with av.open(video) as container:
    stream = container.streams.video[0]
    # We want 2 frame per second
    fps = stream.average_rate
    interval = int(round(fps*0.5))
    #print(fps)
    cpt = 0
    for i,frame in enumerate(container.decode(stream)):
        if i % interval == 0:
            frame.to_image().save(output_folder+"/"+filename+"/"+str(cpt*(0.5))+".jpg", quality=80)
            cpt += 1

## Put the data on opensearch

## Captions generations

## Captions encoding

## Regroupment based on clustering

## Frames fusionning